# @WeRateDogs - Twitter data
Devin de Araujo<br>
25.03.2018
<br>
<img src="tweet_logo.png" width="100" style="float:left">

In [1]:
import pandas as pd
import numpy as np

import requests
import json
import tweepy


# Gathering Data

- Import 'twitter-archive-enhanced.csv' into pandas DataFrame<br>
<br>
- Use Requests to download and save 'image-predictions.tsv'<br>
    - Import this into a pandas DataFrame<br>
<br>
- Use tweepy to get tweet **retweet** and **favorite** counts 


In [ ]:
# download image predictions with requests

url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

r = requests.get(url)

with open('image-predictions.tsv', 'wb') as file:
    file.write(r.content)

In [281]:
# import twitter archive csv and image predictions tsv

twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')

image_predictions = pd.read_csv('image-predictions.tsv', sep= '\t')

In [282]:
# make a copy for later cleaning

twitter_arch = twitter_archive.copy()

image_pred = image_predictions.copy()


In [5]:
twitter_arch.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None


In [6]:
twitter_arch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [271]:
image_pred.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [272]:
image_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


## Twitter API
### Setting up credentials and autherisation

In [9]:
# credentials

consumer_key = 'CONSUMER_KEY'
consumer_secret = 'CONSUMER_SECRET'
access_token = 'ACCESS_TOKEN'
access_secret = 'ACCESS_SECRET'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)


## Getting the Tweets
### Using the get_status() method

1. Create an api object
    - include additional arguments to compensate for api wait time limits
    - <code><b>wait_on_rate_limit</b> = True</code>
    - <code><b>wait_on_rate_limit_notify</b> = True</code><br>
<br>
2. Isolate *tweet* data you want to get<br>
<br>
3. Create a list of **ids** from the **twitter_arch** DataFrame<br>
<br>
4. Using a <code>Loop</code>, iterate through the list to extract data from all tweets
    - append the elements to a blank list<br>
<br>    
5. Add <code>Try-Except Block</code> to handle errors<br>
    - print out errors<br>
<br>
6. Use **pandas** to convert the list into a *DataFrame* <br>
<br>
7. Save the DataFrame to csv

### Tweet exploration

- The below cell contains the initial exploration code I used to extract the tweet data that I potentially wanted to add to my DataFrame
- It has been commented out because does not need to be run, but I wanted to include it for reference purposes

In [ ]:
#### get the following data from the tweet ####

# single_id = '889531135344209921'

# t_list = twitter_arch.tweet_id[0:4]

# api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
# tweet = api.get_status('892177421306343426', tweet_mode='extended')

# tweet Id
# tweet.id

# tweet url
# tweet.entities['media'][0]['url']

# expanded url
# tweet.entities['media'][0]['expanded_url']

# user - favorite_count - THIS IS LIKES
# tweet.favorite_count

# retweet count
# tweet.retweet_count

# get tweet text
#tweet.full_text

# tweet source - currently dirty and unusable without cleaning in twiter enhanced data
# tweet.source

# created time
# this is a datetime object - super duper handy
# you can do created_at.day or .hour etc
# tweet.created_at

# hashtags
#tweet.entities['hashtags'][0]['text']
#tweet.entities['hashtags']

# to convert the tweet to a JSON object you can use a private method called _json
# t = tweet._json

## Extracting tweet info and building a DataFrame

- *source* has been gathered here again because this information is dirty in the 'twitter-archive-enhanced.csv' file
- *text* is also being gathered, just out of interest. It will however, be used from the 'twitter-archive-enhanced.csv' file

In [11]:
# check number of IDs to get a sense of how many rows the query should return

len(twitter_arch.tweet_id[0:])

2356

In [12]:
# create api object
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# subset all tweet ids from twitter archive
all_tweets = twitter_arch.tweet_id[0:]

df_list = []

for item in all_tweets:
    try:
        tweet = api.get_status(item, tweet_mode='extended')
        
        # time to get the goodies !!
        
        tweet_id = tweet.id
        favorite_count = tweet.favorite_count
        retweet_count = tweet.retweet_count
        source = tweet.source
        tweet_text = tweet.full_text
        #hashtags = tweet.entities['hashtags']

        df_list.append({'tweet_id': tweet_id,
                       'source' : source,
                       'favorite_count': int(favorite_count),
                       'retweet_count': int(retweet_count),
                       'tweet_text': str(tweet_text)})
        
    except tweepy.TweepError as e:
        print (e.api_code, e.args[0][0]['message'])

144 No status found with that ID.
144 No status found with that ID.
144 No status found with that ID.
144 No status found with that ID.
144 No status found with that ID.
144 No status found with that ID.
144 No status found with that ID.
144 No status found with that ID.
144 No status found with that ID.
144 No status found with that ID.
144 No status found with that ID.


Rate limit reached. Sleeping for: 480
Rate limit reached. Sleeping for: 467


In [13]:
# Check number of items in the list
len(df_list)

2345

In [135]:
2356 - 2345 # expecting there to be 11 records difference - this accounts for the 11 errors

11

In [300]:
# turn results into a DataFrame

df_tweet_attr = pd.DataFrame(df_list, columns = ['tweet_id', 'source', 'favorite_count',
                                      'retweet_count', 'tweet_text'])

# save the dataframe to csv

df_tweet_attr.to_csv('tweet_attributes.csv', index=False)

df_tweet_attr.head()

,tweet_id,source,favorite_count,retweet_count,tweet_text
0,892420643555336193,Twitter for iPhone,39015,8660,This is Phineas. He's a mystical boy. Only eve...
1,892177421306343426,Twitter for iPhone,33406,6359,This is Tilly. She's just checking pup on you....
2,891815181378084864,Twitter for iPhone,25189,4224,This is Archie. He is a rare Norwegian Pouncin...
3,891689557279858688,Twitter for iPhone,42368,8769,This is Darla. She commenced a snooze mid meal...
4,891327558926688256,Twitter for iPhone,40523,9538,This is Franklin. He would like you to stop ca...


# Assessing the Data

- Detect and document at least **eight (8) quality** issues and **two (2) tidiness** issues
- Identify quality and tidy master pandas DataFrame (or DataFrames, if appropriate).
- Storing, Analysing, and Visualizing Data for this Project
    - Store the clean DataFrame(s) in a CSV file with the main one named **twitter_archive_master.csv.**
    
    
**Key Points**<br>
<br>
Key points to keep in mind when data wrangling for this project:

- You only want **original ratings (no retweets)** that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.<br>
<br>
- Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least **8 quality issues** and at least **2 tidiness** issues in this dataset.<br>
<br>
- Cleaning includes merging individual pieces of data according to the rules of **tidy data**.<br>
<br>
- The fact that the rating numerators are greater than the denominators does not need to be cleaned. This unique rating system is a big part of the popularity of WeRateDogs.<br>
<br>
- You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.

## Quality Issues - twitter-archive-enhanced data

- Any duplicates?
    - Duplicate IDs? Duplicate rows?<br>
<br>
- Html links in text
    - separate these out with **regex**<br>
<br>
- 'None' Values in *doggo, floofer, pupper, puppo & name* columns
    - remove these, replace with empty sting or NaN<br>
<br>
- Remove **retweets**
    - These contain RT in the text and info in retweet columns
    - retweeted_status_id, there are **181 retweets**<br>
<br>
- Remove **replies**
    - These contain **@** in the text and also info in reply columns
    - in_reply_to_status_id, there are **78 replies**<br>
<br>
- Check **Data types**
    - **Timestamp** information incorrect
        - timestamp - object, should be datetime object <br>
<br>
- **name** column quality
    - Missing values (*'none'*)
    - Erroneous names - 55 records with **a** as name
        - there are others - 'Officially', 'the', 'mad' 'they' for example
    - id 776201521193218049 should have **O'Malley** as name<br>
<br>
- **expanded_urls** has multiple **duplicate** urls
    - remove these, keep only one<br>
<br>
- **source** column contains a lot of information
    - in the form of html tags
    - extract the source using **regex**
    - **OR** get this info from the api if it is cleaner

## Quality Issues - image predictions data

- There are incorrect breed names - ie microphone, toy_poodle
- Name capitalisation is inconsistent
- p1_dog, p3_dog, p3_dog columns have FALSE dog records


## Tidiness Issues 
- Tidy up *doggo, floofer, pupper, puppo* columns
    - These contain categorical data types, they are currently messy
    - These can be melted into **one** column<br>
    <br>
- Remove unnecessary columns
    - once **doggo, floofer, pupper, puppo** columns have been corrected
    - once **retweets** and **replies** have been removed <br>
    <br>
- Combine DataFrames into one master
    - **'twitter_archive_master.csv'**
    - Depending on the size of the cleaned **image predictions**
        - If too small, keep separate - not all tweets have predictions

## Assess Image Predicitons

In [161]:
image_pred.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [160]:
print('duplicate tweet_id:', image_pred.tweet_id.duplicated().any(),'\n', 'duplicate records:', image_pred.duplicated().any())

duplicate tweet_id: False 
 duplicate records: False


In [273]:
image_pred.p1_dog.value_counts()

True     1532
False     543
Name: p1_dog, dtype: int64

In [165]:
image_pred.p2_dog.value_counts()

True     1553
False     522
Name: p2_dog, dtype: int64

In [166]:
image_pred.p3_dog.value_counts()

True     1499
False     576
Name: p3_dog, dtype: int64

In [169]:
image_pred.query('p1_dog == False')[:5]

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
17,666104133288665088,https://pbs.twimg.com/media/CT56LSZWoAAlJj2.jpg,1,hen,0.965932,False,cock,0.033919,False,partridge,0.000052,False
18,666268910803644416,https://pbs.twimg.com/media/CT8QCd1WEAADXws.jpg,1,desktop_computer,0.086502,False,desk,0.085547,False,bookcase,0.079480,False
21,666293911632134144,https://pbs.twimg.com/media/CT8mx7KW4AEQu8N.jpg,1,three-toed_sloth,0.914671,False,otter,0.015250,False,great_grey_owl,0.013207,False


There are **False** records in all 3 prediction columns. These contain erroneous dog breeds - it mud_turtle, otter and desk

## Assess Twitter Archive Enhanced

In [18]:
twitter_arch.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None


In [19]:
twitter_arch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [209]:
# check for duplicate values

dup_rec = twitter_arch.duplicated().any() # no duplicate rows

dup_id = twitter_arch.tweet_id.duplicated().any() # no duplicate tweet_ids

unique = twitter_arch.tweet_id.nunique() # can confirm there are only unique ids

print('duplicates records: ',dup_rec, '\n', 'duplicate row: ',dup_id, '\n', 'unique ids: ',unique)

duplicates records:  False 
 duplicate row:  False 
 unique ids:  2356


In [21]:
# check number of retweets and number of replies

print('retweets:', twitter_arch.retweeted_status_id.count(),"\n",'replies:', twitter_arch.in_reply_to_status_id.count())

retweets: 181 
 replies: 78


In [22]:
# check data types
twitter_arch.dtypes

tweet_id                        int64
in_reply_to_status_id         float64
in_reply_to_user_id           float64
timestamp                      object
source                         object
text                           object
retweeted_status_id           float64
retweeted_status_user_id      float64
retweeted_status_timestamp     object
expanded_urls                  object
rating_numerator                int64
rating_denominator              int64
name                           object
doggo                          object
floofer                        object
pupper                         object
puppo                          object
dtype: object

In [23]:
twitter_arch.name.value_counts() #there are columns with 'None' and errors in names (55 with a for name)

# check if there are any more 1 letter names
mask = (twitter_arch['name'].str.len() == 1)
twitter_arch_mask = twitter_arch.loc[mask]
twitter_arch_mask.name.value_counts()

a    55
O     1
Name: name, dtype: int64

In [24]:
# check the O name
twitter_arch_mask.query('name == "O"') # should be O'Malley

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
775,776201521193218049,NaN,NaN,2016-09-14 23:30:38 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is O'Malley. That is how he sleeps. Doesn...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/776201521...,10,10,O,None,None,None,None


In [212]:
# check for other strange names

twitter_arch.name.sort_values()

1035            Abby
1021            Abby
938              Ace
1933            Acro
1327           Adele
1934           Aiden
77               Aja
480            Akumi
820               Al
875           Albert
1954          Albert
144            Albus
412            Albus
1115         Aldrick
2046       Alejandro
374        Alexander
1189    Alexanderson
486              Alf
2238           Alfie
661            Alfie
367            Alfie
1616           Alfie
858            Alfie
51              Alfy
201            Alice
1701           Alice
1495           Amber
1334         Ambrose
2078             Amy
2195          Amélie
            ...     
852               my
988              not
335              not
1747      officially
1206             old
993              one
1936             one
924              one
369              one
2326           quite
169            quite
118            quite
193            quite
2030           space
22              such
2345             the
2346         

In [213]:
# check the value counts for doggo, floofer, pupper, puppo columns

twitter_arch.doggo.value_counts()   # None & doggo
twitter_arch.floofer.value_counts() # None & floofer
twitter_arch.pupper.value_counts()  # None & pupper
twitter_arch.puppo.value_counts()   # None & poppo

# these are all okay, no incorrect category labels

None     2326
puppo      30
Name: puppo, dtype: int64

In [25]:
# use visual assessment (looked through csv in jupyter lab) to find these ids
twitter_arch.query('tweet_id == "768970937022709760"')
twitter_arch.query('tweet_id == "857029823797047296"')

# check the source column
twitter_arch.source[828]
twitter_arch.source[181]

print(twitter_arch.source[828],'\n',twitter_arch.source[181])

<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a> 
 <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>


# Cleaning - image predicitons

- Use <code>.replace()</code> and <code>.title()</code> methods on names to clean and keep consistent
<br>
- Remove any FALSE records from **p1_dog, p2_dog,p3_dog** columns
    - Remove incorrect breed names

In [283]:
# clean breed names
image_pred.p1 = image_pred.p1.str.replace('_', ' ').str.title()
image_pred.p2 = image_pred.p1.str.replace('_', ' ').str.title()
image_pred.p3 = image_pred.p1.str.replace('_', ' ').str.title()

image_pred.p1.value_counts()

Golden Retriever             150
Labrador Retriever           100
Pembroke                      89
Chihuahua                     83
Pug                           57
Chow                          44
Samoyed                       43
Toy Poodle                    39
Pomeranian                    38
Cocker Spaniel                30
Malamute                      30
French Bulldog                26
Miniature Pinscher            23
Chesapeake Bay Retriever      23
Seat Belt                     22
Staffordshire Bullterrier     20
German Shepherd               20
Siberian Husky                20
Web Site                      19
Cardigan                      19
Shetland Sheepdog             18
Beagle                        18
Eskimo Dog                    18
Teddy                         18
Maltese Dog                   18
Shih-Tzu                      17
Lakeland Terrier              17
Rottweiler                    17
Italian Greyhound             16
Kuvasz                        16
          

In [292]:
# drop and records from DF where p1_dog, p2_dog, p3_dog == False

image_pred = image_pred[image_pred['p1_dog'] != False]
image_pred = image_pred[image_pred['p2_dog'] != False]
image_pred = image_pred[image_pred['p3_dog'] != False]

# remove toy poodle !!!
image_pred = image_pred[image_pred['p1'] != 'Toy Poodle']

print('p1',image_pred.p1_dog.value_counts(),'\n', 'p2', image_pred.p2_dog.value_counts(),'\n', 'p3', image_pred.p3_dog.value_counts())

p1 True    1216
Name: p1_dog, dtype: int64 
 p2 True    1216
Name: p2_dog, dtype: int64 
 p3 True    1216
Name: p3_dog, dtype: int64


In [295]:
image_pred.query('p1 == "Toy Poodle"') # NONE
image_pred.query('p1 == "Toy Poodle"') # NONE
image_pred.query('p1 == "Toy Poodle"') # NONE

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog


In [296]:
# save the image_pred df 

image_pred.to_csv('image_predictions_clean.csv', index=False)

# Cleaning - twitter archive data

- Split text and html links from 'text' column<br>
<br>
- Remove "None" values from *doggo, floofer, pupper, puppo, name columns*<br>
<br>
- Drop any retweet records<br>
<br>
- Drop any reply records<br>
<br>
- Turn timestamp into datetime object<br>
<br>
- Change *id 776201521193218049* should have **O'Malley** as name<br>
<br>
- Do some name cleaning<br>
<br>
- Keep only 1 url from *expanded_urls* column<br>
<br>
- Get the *Source* information from twitter API - Drop original source column<br>

## Extract URL from tweet text

In [26]:
# extracts the text and rating

twitter_arch['text_only'] = twitter_arch['text'].str.extract('(\w+.+\d\d.\d\d)', expand=False) 

# extracts the url

twitter_arch['url'] = twitter_arch['text'].str.extract('(https://+\w.+.)', expand=False) 

twitter_arch.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,text_only,url
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None,This is Phineas. He's a mystical boy. Only eve...,https://t.co/MgUWQ76dJU
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None,This is Tilly. She's just checking pup on you....,https://t.co/0Xxu71qeIV


### Drop any retweet and reply records<br>

In [27]:
twitter_arch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 19 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
text_only                     18

In [28]:
# get INDEX of rows you want to drop

reply_index = twitter_arch.query('in_reply_to_status_id == in_reply_to_status_id').index
retweet_index = twitter_arch.query('retweeted_status_id == retweeted_status_id').index

# drop reply and retweet records

twitter_arch.drop(twitter_arch.index[reply_index], inplace = True)
twitter_arch.drop(twitter_arch.index[retweet_index], inplace = True)

# remove those columns

twitter_arch = twitter_arch.drop(['in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id', 
                                  'retweeted_status_user_id','retweeted_status_timestamp'], axis = 1)

twitter_arch.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 14 columns):
tweet_id              2097 non-null int64
timestamp             2097 non-null object
source                2097 non-null object
text                  2097 non-null object
expanded_urls         2093 non-null object
rating_numerator      2097 non-null int64
rating_denominator    2097 non-null int64
name                  2097 non-null object
doggo                 2097 non-null object
floofer               2097 non-null object
pupper                2097 non-null object
puppo                 2097 non-null object
text_only             1657 non-null object
url                   2049 non-null object
dtypes: int64(3), object(11)
memory usage: 245.7+ KB


In [219]:
# check that correct amount of rows were dropped
# 181 retweets
# 78 replies

2356 - (181 + 78)

2097

### Column and value cleaning

- Change *id 776201521193218049* - should have **O'Malley** as name<br>
<br>
- Remove "None" values from **doggo, floofer, pupper, puppo & name** columns<br>
<br>
- Perform some additional name cleaning to remove dirty data
    - look for incorrect names ie. 'the', 'getting', 'not' etc

In [29]:
#twitter_arch.query('tweet_id == "776201521193218049"')['name'] # find the id number

twitter_arch.iloc[[775],[12]] =  "O'Malley"
twitter_arch.iloc[[775],[12]]

,text_only
984,O'Malley


In [30]:
# use str.replace to replace None with blank string
twitter_arch.name = twitter_arch.name.str.replace('None','')

twitter_arch.doggo = twitter_arch.doggo.str.replace('None', '')
twitter_arch.floofer = twitter_arch.floofer.str.replace('None', '')
twitter_arch.pupper = twitter_arch.pupper.str.replace('None', '')
twitter_arch.puppo = twitter_arch.puppo.str.replace('None','')

# check columns visually
twitter_arch[['name','doggo', 'floofer', 'pupper','puppo']][0:7]

,name,doggo,floofer,pupper,puppo
0,Phineas,,,,
1,Tilly,,,,
2,Archie,,,,
3,Darla,,,,
4,Franklin,,,,
5,,,,,
6,Jax,,,,


In [31]:
# some additional name cleaning
# Names are pretty dirty, the trouble with str replace
# is that if there is a name like theo but an incorrect
# name like 'the', then replacing for 'the' will leave
# 'o' in theo's name field

twitter_arch.name = twitter_arch.name.str.replace('officially','', case=True)
twitter_arch.name = twitter_arch.name.str.replace('quite','', case=True)
twitter_arch.name = twitter_arch.name.str.replace('space','', case=True)
twitter_arch.name = twitter_arch.name.str.replace('such','', case=True)
twitter_arch.name = twitter_arch.name.str.replace('this','', case=True)
twitter_arch.name = twitter_arch.name.str.replace('unacceptable','', case=True)
twitter_arch.name = twitter_arch.name.str.replace('very','', case=True)
twitter_arch.name = twitter_arch.name.str.replace('my','', case=True)
twitter_arch.name = twitter_arch.name.str.replace('incredibly','', case=True)
twitter_arch.name = twitter_arch.name.str.replace('infuriating','', case=True)
twitter_arch.name = twitter_arch.name.str.replace('just','', case=True)
twitter_arch.name = twitter_arch.name.str.replace('light','', case=True)
twitter_arch.name = twitter_arch.name.str.replace('getting','', case=True)
twitter_arch.name = twitter_arch.name.str.replace('life','', case=True)
twitter_arch.name = twitter_arch.name.str.replace('actually','', case=True)
twitter_arch.name = twitter_arch.name.str.replace('old','', case=True)
twitter_arch.name = twitter_arch.name.str.replace('his','', case=True)
twitter_arch.name = twitter_arch.name.str.replace('one','', case=True)
twitter_arch.name = twitter_arch.name.str.replace('not','', case=True)
twitter_arch.name = twitter_arch.name.str.replace('the','', case=True)

#twitter_arch.name.sort_values()

### Keep only 1 url from *expanded_urls* column

In [32]:
twitter_arch['expanded_urls_trim'] = twitter_arch.expanded_urls.str.extract('(https://twitter.com/dog_rates/status/\d+)', expand=True)

twitter_arch.head(2)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,text_only,url,expanded_urls_trim
0,892420643555336193,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,,,,,This is Phineas. He's a mystical boy. Only eve...,https://t.co/MgUWQ76dJU,https://twitter.com/dog_rates/status/892420643...
1,892177421306343426,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,,,,,This is Tilly. She's just checking pup on you....,https://t.co/0Xxu71qeIV,https://twitter.com/dog_rates/status/892177421...


### Clean timestamp column
- datatype as datetime object

In [33]:
type(twitter_arch['timestamp'][0])

str

In [34]:
twitter_arch['timestamp'] = pd.to_datetime(twitter_arch.timestamp, format = '%Y/%m/%d %H:%M:%S')
type(twitter_arch['timestamp'][0])

pandas._libs.tslib.Timestamp

In [35]:
# check if it worked
twitter_arch['timestamp'][0].month

8

# Tidy the data

### Drop unnecessary columns
- 'text','expanded_urls', 'source' - Keep cleaned columns
    - 'source' was gathered much cleaner from the api
        - it was easier to do this, and saved more time than it would have taken to clean the column
- melt *doggo, floofer, pupper, puppo* columns  

In [36]:
twitter_arch = twitter_arch.drop(['text','expanded_urls','source'], axis=1)

twitter_arch = twitter_arch[['tweet_id','timestamp','text_only','url',
                             'expanded_urls_trim','rating_numerator',
                            'rating_denominator','name', 'doggo', 'floofer', 'pupper', 'puppo']]
twitter_arch.head(2)

,tweet_id,timestamp,text_only,url,expanded_urls_trim,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,2017-08-01 16:23:56,This is Phineas. He's a mystical boy. Only eve...,https://t.co/MgUWQ76dJU,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,,,,
1,892177421306343426,2017-08-01 00:17:27,This is Tilly. She's just checking pup on you....,https://t.co/0Xxu71qeIV,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,,,,
2,891815181378084864,2017-07-31 00:18:03,This is Archie. He is a rare Norwegian Pouncin...,https://t.co/wUnZnhtVJB,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,,,,
3,891689557279858688,2017-07-30 15:58:51,This is Darla. She commenced a snooze mid meal...,https://t.co/tD36da7qLQ,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,,,,
4,891327558926688256,2017-07-29 16:00:24,This is Franklin. He would like you to stop ca...,https://t.co/AtUZn91f7f,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,,,,


In [43]:
# rename some columns

twitter_arch.columns = ['tweet_id', 'timestamp', 'text', 'url', 'expanded_urls','rating_numerator','rating_denominator','name', 'doggo', 'floofer', 'pupper','puppo']

twitter_arch.head(2)

,tweet_id,timestamp,text,url,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,2017-08-01 16:23:56,This is Phineas. He's a mystical boy. Only eve...,https://t.co/MgUWQ76dJU,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,,,,
1,892177421306343426,2017-08-01 00:17:27,This is Tilly. She's just checking pup on you....,https://t.co/0Xxu71qeIV,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,,,,


### Re-shape doggo, floofer, pupper, puppo columns

In [170]:
twitter_clean = pd.melt(twitter_arch, id_vars = ['tweet_id','timestamp','text','url',
                             'expanded_urls','rating_numerator','rating_denominator','name'],
                       var_name = 'variable', value_name = 'type')

# drop the variable column - don't need this
twitter_clean = twitter_clean.drop('variable', axis = 1)

# sort the data
twitter_clean = twitter_clean.sort_values(by='type')

# drop any duplicate ids - this will remove 'doggopupper' dirty records
twitter_clean = twitter_clean.drop_duplicates(subset=['tweet_id'],keep='first')

twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 9 to 2096
Data columns (total 9 columns):
tweet_id              2097 non-null int64
timestamp             2097 non-null object
text                  1657 non-null object
url                   2049 non-null object
expanded_urls         1947 non-null object
rating_numerator      2097 non-null int64
rating_denominator    2097 non-null int64
name                  1433 non-null object
type                  340 non-null object
dtypes: int64(3), object(6)
memory usage: 163.8+ KB


In [171]:
# check the value counts
twitter_clean.type.value_counts()

pupper     229
doggo       78
puppo       24
floofer      9
Name: type, dtype: int64

In [173]:
twitter_clean.duplicated(subset='tweet_id').any()

False

### Save the new df

In [174]:
twitter_clean.to_csv('twitter_archive_clean.csv', index = False)

### Combine tweet attributes Data Frame

In [327]:
# import the twitter attributes df. Don't need the text field since we already have this

fields = ['tweet_id', 'source', 'favorite_count', 'retweet_count']

tweet_att = pd.read_csv('tweet_attributes.csv', usecols=fields)

tweet_att.head()

,tweet_id,source,favorite_count,retweet_count
0,892420643555336193,Twitter for iPhone,39015,8660
1,892177421306343426,Twitter for iPhone,33406,6359
2,891815181378084864,Twitter for iPhone,25189,4224
3,891689557279858688,Twitter for iPhone,42368,8769
4,891327558926688256,Twitter for iPhone,40523,9538


In [176]:
tweet_att.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2345 entries, 0 to 2344
Data columns (total 4 columns):
tweet_id          2345 non-null int64
source            2345 non-null object
favorite_count    2345 non-null int64
retweet_count     2345 non-null int64
dtypes: int64(3), object(1)
memory usage: 73.4+ KB


In [328]:
combine_df = pd.merge(twitter_clean, tweet_att, on='tweet_id', how='left')

combine_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2096
Data columns (total 12 columns):
tweet_id              2097 non-null int64
timestamp             2097 non-null object
text                  1657 non-null object
url                   2049 non-null object
expanded_urls         1947 non-null object
rating_numerator      2097 non-null int64
rating_denominator    2097 non-null int64
name                  1433 non-null object
type                  340 non-null object
source                2088 non-null object
favorite_count        2088 non-null float64
retweet_count         2088 non-null float64
dtypes: float64(2), int64(3), object(7)
memory usage: 213.0+ KB


In [330]:
# check for any duplicate values
combine_df.duplicated(subset='tweet_id').any()

False

I am not going to combine the image-predictions DataFrame because this DataFrame is not as clean as the tweet_attributes and the twitter-archive-enhanced data and is (in my opinion) not very robust. This data frame is also the smallest, meaning there would me a lot of missing information, since not all tweets have image predictions. Tables can be joined later on tweet_id if any image analysis is to be done.

### Store the combined df as twitter_archive_master.csv

In [329]:
combine_df.to_csv('twitter_archive_master.csv', index = False)

## The visualizations are in the act_report.ipynd notebook